# Resumo com modelo do HuggingFace

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

In [ ]:
hub = {
	'HF_MODEL_ID':'GiordanoB/mT5_multilingual_XLSum-sumarizacao-PTBR',
	'HF_TASK':'summarization'
}

huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

In [ ]:
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type='ml.m5.xlarge' # ou ml.c5.large, ml.m4.xlarge
)

In [ ]:
predictor.predict({
	'inputs': " recuperação da economia veio forte para a indústria de transformação, com nove altas seguidas nas horas trabalhadas na produção, o que levou o indicador ao maior nível desde o fim de 2015. Em fevereiro de 2021, contudo, essa sequência de altas foi interrompida com uma queda de 0,5%. O desaquecimento da atividade industrial também provocou retração do faturamento (-3,3%), da massa salarial (-1,1%), do rendimento médio (-1,8%) e da utilização da capacidade instalada (-0,4 ponto percentual)."
})

# Efetuando chamados para o endpoint utilizando o Boto3

In [ ]:
endpoint = predictor.endpoint

In [ ]:
import boto3
import json
import sagemaker

# create a boto3 client for the SageMaker runtime
client = boto3.client('sagemaker-runtime')

model_package_arn = endpoint
role = sagemaker.get_execution_role()



# set the content type and accept header for the request
content_type = "application/json"
accept = "application/json"

# create the request payload
payload = {'inputs': """

As tecnologias on-line está transformando o mundo, e as universidades não são exceção. Estima-se que 50% das universidades terão implementado alguma tecnologia on-line até 2025. A tecnologia está mudando a forma como as instituições de ensino superior conduzem pesquisas, oferecem aulas e interagem com seus alunos e comunidades. 

O principal desafio de Brazil é democratizar o acesso a tecnologias disruptivas evitando a concentração da adoção apenas em grandes capitais como Sao Paulo. Essa vem sendo uma das prioridades do ministro da Educação Camilo Santana. Ele destacou a liderança do grupo Cogna como exemplo de democratização do ensino.

O ministro salientou nesta quinta-feira que as universidades podem fornecer uma experiência de aprendizagem mais personalizada para seus alunos analisando dados de desempenho dos alunos e fornecer feedback personalizado e recomendações para melhorias. Isso ajuda os alunos a entenderem melhor seus pontos fortes e fracos e a adaptarem sua aprendizagem de acordo.

Ele também citou que a pesquisa científica também está sendo transformada. Os algoritmos podem analisar grandes conjuntos de dados e identificar padrões e tendências que podem levar a novas descobertas e avanços. Isso ajuda os pesquisadores a economizar tempo e recursos, bem como a aumentar a precisão e a eficácia de suas pesquisas.

Por fim o ministro enalteceu que as ferramentas de colaboração também está tornando-se mais fáceis entre professores e alunos. As ferramentas de inteligencia artificial podem ajudar a identificar áreas de interesse comum entre os membros da equipe e facilitar a comunicação e o compartilhamento de recursos. 

O ministro Camilo Santana fechou o seu discurso salientando a tecnologias disruptivas vão mudar a forma com que as universidades levm conteúdo aos seus alunos, permitindo que cursos online levem o estado da arte em programas de graduação que podem ser acessados de qualquer lugar do mundo. O uso de tecnologia vem sendo fundamental para a educação fornecendo oportunidades para a população.

"""}

predictor = sagemaker.predictor.Predictor(
    model_package_arn,
    role=role,
    sagemaker_session=sagemaker.Session()
)

# invoke the endpoint and capture the response
response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType=content_type,
    Accept=accept,
    Body=json.dumps(payload)
)


# extract the predicted output from the response
result = json.loads(response['Body'].read().decode())

print(result)